### Data preparation and preprocessing

In [3]:
import cv2
import os

def extract_frames(video_path, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Open the video file
    vidcap = cv2.VideoCapture(video_path)
    success, image = vidcap.read()
    count = 0

    # Loop through the video frames
    while success:
        # Save the frame as a JPEG file
        frame_path = os.path.join(output_folder, f"frame{count}.jpg")
        cv2.imwrite(frame_path, image)
        
        # Read the next frame
        success, image = vidcap.read()
        count += 1

    print(f"Extracted {count} frames from {video_path} to {output_folder}")

# Example usage
video_path = r'C:\Users\Hassan\Desktop\vid.mp4'
output_folder = 'output_frames'
extract_frames(video_path, output_folder)


Extracted 1500 frames from C:\Users\Hassan\Desktop\vid.mp4 to output_frames


#### Face extraction

In [4]:
import cv2
import os

def extract_faces(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        image = cv2.imread(img_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w]
            face_path = os.path.join(output_folder, filename)
            cv2.imwrite(face_path, face)

# Example usage
input_folder = 'output_frames'
output_folder = 'faces'
extract_faces(input_folder, output_folder)

#### Resizing frames

In [5]:
from PIL import Image

def resize_images(input_folder, output_folder, size=(224, 224)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        image = Image.open(img_path)
        image = image.resize(size)
        image.save(os.path.join(output_folder, filename))

# Example usage
input_folder = 'faces'
output_folder = 'resized_faces'
resize_images(input_folder, output_folder)

#### Normalize the frames

In [6]:
import numpy as np

def normalize_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        img_path = os.path.join(input_folder, filename)
        image = cv2.imread(img_path)
        image = image / 255.0  # Normalize pixel values
        normalized_path = os.path.join(output_folder, filename)
        cv2.imwrite(normalized_path, (image * 255).astype(np.uint8))

# Example usage
input_folder = 'resized_faces'
output_folder = 'normalized_faces'
normalize_images(input_folder, output_folder)

### Labelling the frames

In [7]:
import csv
import os

# Paths
input_folder = r'C:\Users\Hassan\Desktop\ML_DL\myenv\normalized_faces'  # Folder with normalized frames
output_csv = r'C:\Users\Hassan\Desktop\ML_DL\myenv\labels.csv'  # CSV file to save labels

# Create/Open the CSV file for writing
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['frame_path', 'label'])  # Write header

    # Iterate through all images in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(input_folder, filename)
            frame_number = int(filename.split('frame')[1].split('.')[0])  # Extract frame number
            
            # Label based on the provided ranges
            if 0 <= frame_number <= 269:
                label = 0  # Low Stress
            elif 270 <= frame_number <= 313:
                label = 1  # Medium Stress
            elif 314 <= frame_number <= 656:
                label = 0  # Low Stress
            elif 657 <= frame_number <= 676:
                label = 1  # Medium Stress
            elif 677 <= frame_number <= 730:
                label = 2  # High Stress
            elif 731 <= frame_number <= 750:
                label = 1  # Medium Stress
            elif 809 <= frame_number <= 956:
                label = 2  # High Stress
            elif 991 <= frame_number <= 1003:
                label = 2  # High Stress
            else:
                label = ''  # Validation set (leave blank)

            # Write the image path and label to CSV
            writer.writerow([img_path, label])

In [27]:
import pandas as pd

# Load the original CSV file
csv_file = 'labels.csv'
df = pd.read_csv(csv_file)

# Sort the DataFrame
df_sorted = df.sort_values(by=['label'], na_position='last')

# Save the sorted DataFrame to a new CSV file
sorted_csv_file = 'sorted_labels.csv'
df_sorted.to_csv(sorted_csv_file, index=False)

print(f"Sorted CSV file saved to {sorted_csv_file}")

Sorted CSV file saved to sorted_labels.csv


#### Splitting dataset into training and validation

In [29]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

# Load the CSV file
data = pd.read_csv(r'C:\Users\Hassan\Desktop\ML_DL\myenv\labels.csv')

# Filter out rows where label is not null
data = data.dropna(subset=['label'])

# Initialize lists to hold images and labels
images = []
labels = []

# Iterate through the data and load images and labels
for index, row in data.iterrows():
    img_path = str(row['frame_path'])  # Ensure path is converted to string
    label = row['label']
    
    if os.path.exists(img_path):  # Check if the path exists
        # Load and preprocess the image
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = image.astype('float32') / 255.0

        images.append(image)
        labels.append(int(label))
    else:
        print(f"File not found: {img_path}")

# Convert to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Split into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Ensure we have non-empty val_labels
print(f"Shape of train_images: {train_images.shape}")
print(f"Shape of val_images: {val_images.shape}")
print(f"Validation labels (before): {val_labels.shape}")

# Correct the validation labels assignment
val_labels = val_labels[:len(val_images)]

print(f"Validation labels (after): {val_labels.shape}")

Shape of train_images: (425, 224, 224, 3)
Shape of val_images: (107, 224, 224, 3)
Validation labels (before): (107,)
Validation labels (after): (107,)


### Training - PHASE 1 (CNN's)

In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [31]:
# Define the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')  # Assuming 3 output classes (0, 1, 2)
])

# Compile the mode
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display model architecture
model.summary()

C:\Users\Hassan\Desktop\ML_DL\myenv\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,680,067 (36.93 MB)

 Trainable params: 9,680,067 (36.93 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Train the model
history = model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_images, val_labels)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4825 - loss: 1.1252 - val_accuracy: 0.6355 - val_loss: 0.6935
Epoch 2/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.7397 - loss: 0.5354 - val_accuracy: 0.9065 - val_loss: 0.1972
Epoch 3/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.8836 - loss: 0.3190 - val_accuracy: 0.8785 - val_loss: 0.2679
Epoch 4/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.9119 - loss: 0.2075 - val_accuracy: 0.9626 - val_loss: 0.1153
Epoch 5/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.9194 - loss: 0.2230 - val_accuracy: 0.9159 - val_loss: 0.2046
Epoch 6/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.9277 - loss: 0.1740 - val_accuracy: 0.9252 - val_loss: 0.1622
Epoch 7/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.9312 - loss: 0.1832 - val_accuracy: 0.9439 - val_loss: 0.1012
Epoch 8/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.9587 - loss: 0.1174 - val_accuracy: 0.9439 - val_loss:

### Optimizing Baseline Model

#### Using face ROI Detection method for face detection (instead of openCV)

In [ ]:
# import dlib
# import cv2
# import os
# import numpy as np
# from tensorflow.keras.models import load_model

# # Initialize Dlib's face detector
# detector = dlib.get_frontal_face_detector()

# # Function to detect faces and return the ROIs
# def detect_faces_dlib(image):
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
#     faces = detector(gray)  # Detect faces
#     rois = [image[d.top():d.bottom(), d.left():d.right()] for d in faces]  # Extract face ROIs
#     return rois

# # Load your trained model
# model = load_model('Desktop/ML_DL/my_model.keras')

# # Function to preprocess the face for the model
# def preprocess_face(face):
#     face = cv2.resize(face, (224, 224))  # Resize to the model's input size
#     face = face.astype('float32') / 255.0  # Normalize pixel values
#     face = np.expand_dims(face, axis=0)  # Add batch dimension
#     return face

# # Path to the folder containing frames
# frames_folder = 'output_frames'

# # Iterate through each frame
# for filename in os.listdir(frames_folder):
#     img_path = os.path.join(frames_folder, filename)
#     image = cv2.imread(img_path)
#     rois = detect_faces_dlib(image)
    
#     for i, roi in enumerate(rois):
#         preprocessed_face = preprocess_face(roi)
#         predictions = model.predict(preprocessed_face)
#         predicted_label = np.argmax(predictions, axis=1)[0]

#         # Map predicted label to stress level
#         stress_levels = {0: 'Low Stress', 1: 'Medium Stress', 2: 'High Stress'}
#         predicted_stress_level = stress_levels[predicted_label]
        
#         # Print or save the result
#         print(f"Frame {filename} - Face {i+1}: {predicted_stress_level}")
#         cv2.imshow(f'Face {i+1} - {predicted_stress_level}', roi)
#         cv2.waitKey(1000)  # Display for 1 second

# cv2.destroyAllWindows()

In [ ]:
# pip install dlib